In [12]:
%matplotlib inline
import torch
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

In [13]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [14]:
batch_size = 10

In [15]:
# torch_1
import torch.utils.data as Data
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset,batch_size,shuffle=True)

for X,y in data_iter:
    print(X,y)
    break

tensor([[ 0.3678, -0.2020],
        [ 0.7015,  0.1579],
        [ 2.6637, -0.0442],
        [ 0.2539,  1.0343],
        [-1.5512,  0.1465],
        [-0.4198,  0.1710],
        [ 0.7243,  0.6127],
        [-2.1002,  0.0613],
        [ 0.7158,  0.0743],
        [ 0.1551, -0.4660]]) tensor([ 5.6194,  5.0792,  9.6775,  1.1820,  0.5894,  2.7863,  3.5651, -0.2236,
         5.3710,  6.0979])


In [16]:
import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet,self).__init__()
        self.linear = nn.Linear(n_feature,1)
    # forward
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)    #0
print(net)

net = nn.Sequential(           #1
    nn.Linear(num_inputs,1)
)
print(net)

net = nn.Sequential()         #2
net.add_module('linear',nn.Linear(num_inputs,1))
print(net)

from collections import OrderedDict #3
net = nn.Sequential(OrderedDict([
    ('linear',nn.Linear(num_inputs,1))
]))
print(net)

for param in net.parameters():
    print(param)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)
Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Parameter containing:
tensor([[0.6249, 0.0886]], requires_grad=True)
Parameter containing:
tensor([0.1246], requires_grad=True)


In [17]:
# torch_2
net = nn.Sequential()         #2
net.add_module('linear',nn.Linear(num_inputs,1))

from torch.nn import init

init.normal_(net[0].weight,mean=0,std=0.01)
init.constant_(net[0].bias,val=0)

Parameter containing:
tensor([0.], requires_grad=True)

In [18]:
# torch_3
loss = nn.MSELoss()

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [19]:
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

In [20]:
# torch_4
num_epochs = 3
for epoch in range(1,num_epochs+1):
    for X,y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1,1))
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 21.526468
epoch 2, loss: 1.642940
epoch 3, loss: 1.231167


In [21]:
dense = net[0]
print(true_w,dense.weight)
print(true_b,dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.7299, -2.8442]], requires_grad=True)
4.2 Parameter containing:
tensor([3.4965], requires_grad=True)
